In [1]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, LLMPredictor, ServiceContext, StorageContext, load_index_from_storage, Document
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.indices.postprocessor import SentenceTransformerRerank

import os
from langchain import HuggingFaceHub
import sys
import logging
import torch

sys.path.append('../')
from apikeys import huggingface_key
import tqdm as notebook_tqdm

In [2]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = huggingface_key

In [4]:
from llama_index.llms import HuggingFaceInferenceAPI
mixtral = HuggingFaceInferenceAPI(
    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
)

In [5]:
# load the documents and create the index
documents = SimpleDirectoryReader("../data").load_data()
document = Document(text="\n\n".join([doc.text for doc in documents]))

In [6]:
from llama_index.llms import ChatMessage, MessageRole
from llama_index.prompts import ChatPromptTemplate

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "Always answer the question, even if the context isn't helpful."
        ),
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information and not prior knowledge, "
            "answer the question: {query_str}\n"
        ),
    ),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

# Refine Prompt
chat_refine_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "Always answer the question, even if the context isn't helpful."
        ),
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "We have the opportunity to refine the original answer "
            "(only if needed) with some more context below.\n"
            "------------\n"
            "{context_msg}\n"
            "------------\n"
            "Given the new context, refine the original answer to better "
            "answer the question: {query_str}. "
            "If the context isn't useful, output the original answer again.\n"
            "Original Answer: {existing_answer}"
        ),
    ),
]
refine_template = ChatPromptTemplate(chat_refine_msgs)

In [7]:
def build_sentence_window_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index",
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            documents, service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index


def get_sentence_window_query_engine(
    sentence_index, similarity_top_k=6, rerank_top_n=2
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank], text_qa_template=text_qa_template, refine_template=refine_template
    )
    return sentence_window_engine

In [8]:
index = build_sentence_window_index(
    [document],
    llm=mixtral,
    save_dir="./sentence_index",
    sentence_window_size = 3
)
query_engine = get_sentence_window_query_engine(index, similarity_top_k=6)

In [9]:
eval_questions = [
    "when should you worry about baby split up?",
    "how often should you feed your baby?",
    "how much sleep should your baby get?",
    "how often should a baby poop?",
    "how should a clean my baby?"
]

In [10]:
for question in eval_questions:
    print(
        query_engine.query(question)
    )

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


You should worry about your baby's breathing if it becomes fast and noisy, or if there's a sudden worsening of your baby's cold symptoms or wheezing. Also, if your baby's cough becomes croupy, which is a deep, barking cough, or if your baby seems to be becoming dehydrated, these are signs that you should seek medical attention. 

In the context of constipation, you should be concerned if your baby is having difficulty or discomfort passing hard, dry stools. If your baby hasn't had a bowel movement in a few days, or if you notice that your baby is straining or crying during bowel movements, these could be signs of constipation. 

In both cases, it's important to consult with a healthcare professional to ensure your baby's health and well-being.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Based on the provided context, babies, whether breast-fed or formula-fed, will generally need feeding every 2 to 3 hours.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]



A normal newborn will sleep about 16 -20 hours a day, and around 3 months of age and 12 pounds, the baby is developmentally ready to find their own comfy spot for 10-12 hours of sleep in a row.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]



Based on the context information provided, the frequency of a baby's poop can vary greatly depending on their diet and age. For breastfed babies, it's normal for them to poop less frequently after the first few months, sometimes only once every 4 or 5 days. However, these stools should still be soft and mushy. For formula-fed babies, the stool can be darker in color and may vary from yellow to brown to green. The frequency can also vary, but in the first 2-3 weeks, they may poop every feeding or as infrequently as once a week. If a baby is not pooping frequently and is showing signs of discomfort, it could be a sign of not getting enough milk or a reaction to certain medications.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]



To clean your baby, start by washing their body from top to bottom, cleaning the diaper area last. Make sure to clean well between the skin folds. Use a clean washcloth and no soap to wash your baby’s face, gently wiping from the inside to the outside corner of each eye. You can use clean water and a clean washcloth or cotton balls for this. 

The baby's scalp does not need to be cleaned more than twice a week. Parents are advised to be judicious in their use of skin care products in young babies. Powders and lotion are unnecessary. If commercial baby wipes used for diaper change cause irritation, use plain water to clean your baby. 

After the baby's bath, pat him thoroughly dry. Once the cord has come off and the circumcision site of a boy has healed, the baby can have a real bath.
